# Unit 3 Developing a MCP Server for a Shopping List Service

Welcome back\! In the last lesson, you learned how to define and expose the main building blocks of an MCP server: **tools**, **resources**, and **prompts**. You saw how to make these primitives discoverable and usable by clients, and you practiced connecting to your server and interacting with each type. This lesson will build directly on that foundation.

In this lesson, you will learn how to take a real-world service — in this case, a **shopping list manager** — and expose its actions as MCP tools. You will see how to wrap service methods as tools, document them clearly, and make them available for clients to discover and use. By the end of this lesson, you will be able to define, document, and implement practical MCP tools for a service, and you will know how to connect to your server and interact with these tools manually. This will prepare you for more advanced integrations, such as connecting your server to an AI agent, which we will cover in the next lesson.

## Overview Of The Shopping List Service

To make this lesson practical, we will use a simple **shopping list service** as our example. The shopping list service is implemented in a file called `shopping_list.py`. This service manages a collection of shopping items, each with an **ID**, **name**, **quantity**, and **purchased** status. The service provides methods to add items, remove items, mark items as purchased or not, and fetch the list of items (optionally filtered by whether they have been purchased).

You do not need to write the shopping list logic yourself — the code is already provided for you. Here is a quick look at the core of the service:

```python
class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""

    def __init__(self):
        # Initialize the shopping list with some default items
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            # ... more items ...
        ]

    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        # ... implementation ...
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # ... implementation ...
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # ... implementation ...
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # ... implementation ...
```

This service is a typical example of a backend component you might want to expose to AI agents or other clients. By wrapping its methods as MCP tools, you make it possible for clients to add, remove, and update shopping list items in a standardized way.

## Setting Up the MCP Server and Shopping List Service

Before you can expose the shopping list service’s actions as MCP tools, you need to set up your MCP server and create an instance of the shopping list service.

```python
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()

# Define MCP tools...

if __name__ == "__main__":
    # Run the server with the desired transport
    mcp.run(transport="stdio")
```

With this setup, your MCP server instance is ready to register tools, and your `shopping_list` service instance is ready to handle shopping list operations.

## Defining And Documenting MCP Tools

To expose the shopping list service’s actions as MCP tools, you will use the `@mcp.tool()` decorator, just as you did in the previous lesson. Each tool is a Python function that calls the appropriate method on the service. It is important to provide clear docstrings for each tool, describing what it does, what parameters it takes, and what it returns. This documentation is not just for humans — it is also used by clients and AI agents to understand how to use your tools.

Let’s start by defining a tool to **fetch shopping list items**. This is a common operation that clients will need, and it demonstrates how to expose a service method as a tool with clear documentation.

```python
@mcp.tool()
def fetch_items(purchased: bool = None) -> list:
    """
    Retrieve all shopping list items, optionally filtered by purchased status.

    Parameters:
        purchased (bool, optional): Filter items by purchased status.
            None returns all items,
            True returns only purchased items,
            False returns only unpurchased items.

    Returns:
        list: A list of shopping items matching the filter criteria.
    """
    return shopping_list.get_items(purchased)
```

Notice how the docstring explains what the tool does, lists the parameters with their types and descriptions, and describes the return value. This level of documentation is essential for making your tools easy to use and understand, especially when clients or AI agents are discovering them dynamically.

### Why Use a Tool Instead of a Resource to Return Data?

In MCP, both **tools** and **resources** can be used to expose information to clients, but they are designed for different scenarios.

**Tools** are **model-controlled**, which means AI agents and clients can invoke them directly and autonomously. Tools can accept parameters, perform logic, and return results dynamically. This makes them ideal for retrieving data that may change over time, needs to be filtered, or requires real-time access. For example, fetching shopping list items—especially when you want to filter by purchased status or always get the most current data—is best implemented as a tool. This allows agents to request exactly the data they need, when they need it, as part of a conversation or workflow.
**Resources** are **application-controlled** and are typically used to provide static or pre-defined data, such as documentation or reference material. The client application decides when and how to use resources, and agents cannot invoke them directly in the same way as tools. This makes resources less suitable for dynamic data retrieval or actions that require parameters.

For the shopping list service, exposing data-fetching as a tool ensures that agents can always retrieve the latest information, apply filters, and interact with the service in a flexible, real-time manner. This is why we use a tool—not a resource—to return shopping list data.

## Implementing The Shopping List Tools

Now, let’s walk through the other main tools you will expose for the shopping list service. Each tool wraps a method from the service and provides a clear interface for clients.

The `add_item` tool allows clients to add a new item to the shopping list. It takes a `name` and `quantity`, and returns the unique ID of the new item.

```python
@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
        name (str): The name of the item.
        quantity (int): The quantity to add.

    Returns:
        str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)
```

The `remove_item` tool lets clients remove an item by its ID. It returns `True` if the item was removed successfully, or `False` if no item with that ID was found.

```python
@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.

    Parameters:
        item_id (str): The unique ID of the item.

    Returns:
        bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)
```

The `mark_purchased` tool allows clients to mark an item as purchased or not purchased. It takes the item’s ID and a boolean flag, and returns `True` if the update succeeded.

```python
@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.

    Parameters:
        item_id (str): The unique ID of the item.
        purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.

    Returns:
        bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)
```

Each of these tools is now discoverable and callable by any MCP client. The clear docstrings make it easy for both humans and AI agents to understand how to use them.

## Connecting to the MCP Server

Once your tools are defined, you can run and connect to your server using the MCP client interface. For local development and testing, the `stdio` transport is commonly used. Here’s a reminder of how to connect to your MCP server:

```python
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            # You can now interact with the server using the session object

if __name__ == "__main__":
    asyncio.run(main())
```

## Listing Available Tools

After connecting to the server, you can discover which tools are available and inspect their documentation and input schemas. This is a crucial step, as it allows you (or any client, including AI agents) to understand what actions the server exposes and how to use them.

To demonstrate this, let’s list all available tools and print out their key details:

```python
# List all tools exposed by the server
tools_list = await session.list_tools()

# For each tool, display its name, description, and input schema
for tool in tools_list.tools:
    print(f"Name: {tool.name}")
    print(f"Description: {tool.description}")
    print(f"Input Schema:\n{tool.inputSchema}\n")
```

Running this code will output something like the following for each tool:

```
Name: add_item
Description: 
    Add a shopping list item.
    
    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.
      
    Returns:
      str: The unique ID of the newly added item.
    
Input Schema:
{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'quantity': {'title': 'Quantity', 'type': 'integer'}}, 'required': ['name', 'quantity'], 'title': 'add_itemArguments', 'type': 'object'}
```

This demonstration shows how you can programmatically explore the server’s capabilities, making it easy to see what actions are available and how to call them.

## Adding and Fetching Shopping List Items

Let’s walk through how to use these tools in practice. First, we’ll add a new item to the shopping list, and then we’ll fetch the updated list to verify the change.

To add an item, call the `add_item` tool with the desired name and quantity:

```python
# Add a new item to the shopping list
response_add = await session.call_tool(
    "add_item",
    {
        "name": "Bananas",
        "quantity": 3
    }
)

# Extract and print the ID of the newly added item
added_id = response_add.content[0].text
print(f"Added item ID: {added_id}")
```

This demonstrates how to invoke a tool and handle its response. The output will look like:

```
Added item ID: 9322bc54-588d-4855-a440-697627e427fc
```

### Fetching the Shopping List Items

Next, let’s fetch the current shopping list items to confirm that the new item was added:

```python
# Retrieve all shopping list items
response_fetch = await session.call_tool("fetch_items", {})

# Print each item in the shopping list
print("Current shopping list items:")
for item in response_fetch.content:
    print(item.text)
```

This will display the full list of items, including the one you just added:

```
Current shopping list items:
{"id": "cb9abd6a-ff42-4808-8552-e664221d5e00", "name": "Milk", "quantity": 2, "purchased": true}
{"id": "06a2fa28-56d9-416a-bc17-3b70bf0107e4", "name": "Bread", "quantity": 1, "purchased": false}
{"id": "59f0f8f2-9309-4a3b-aedb-876faf1653b5", "name": "Eggs", "quantity": 12, "purchased": true}
{"id": "493afcc2-d018-4a9f-b3e7-bf05015b7243", "name": "Apples", "quantity": 6, "purchased": false}
{"id": "8c843bf2-0bb5-402e-85e3-c5aba4e652f4", "name": "Coffee", "quantity": 1, "purchased": false}
{"id": "1e93cfe9-a5f1-483d-b958-09844647e25b", "name": "Bananas", "quantity": 3, "purchased": false}
```

By following these steps, you can see how to discover, call, and verify the behavior of MCP tools in a real-world scenario. This hands-on demonstration shows the full cycle: exploring available actions, invoking them, and checking the results.

## Summary And Preparation For Practice

In this lesson, you learned how to take a real-world service and expose its actions as MCP tools. You saw how to wrap service methods with the `@mcp.tool()` decorator, document them clearly, and handle inputs and outputs in a way that is easy for clients and AI agents to use. You also practiced running the MCP server and connecting to it manually from a client, listing available tools, and calling them with real data.

These skills are essential for building practical, useful MCP servers that can power AI agents and other applications. In the next section, you will get hands-on practice by defining and using your own tools for the shopping list service. This will help you solidify your understanding and prepare you for more advanced integrations in future lessons.

## Creating an MCP Tool for a Service

You’ve just seen how to set up an MCP server and learned how to expose service actions as tools with clear documentation. Now it’s your turn to put this into practice.

Your task is to set up the shopping list service inside the MCP server file and create a tool for adding new items. Follow these steps:

Initialize the ShoppingListService so it’s ready to use.
Define the add_item function and register it as an MCP tool. The function should take two parameters: name (a string representing the item's name) and quantity (an integer representing how many to add).
Write a clear docstring for the tool. Be sure to explain what the tool does, describe its parameters and state what it returns (the new item’s ID).
In the function, call the shopping list service to add the item and return the new item’s ID.
When the server starts, it should print out all registered tools so you can see that your tool is discoverable.

```python
# mcp_server.py

# shopping_list.py

# main.py
```

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# TODO: Initialize the shopping list service


# TODO: Define the add_item function and register it as an MCP tool
    # TODO: Parameters: name (str), quantity (int)
    # TODO: Write a docstring that explains what this tool does, its parameters, and what it returns
    # TODO: Call the shopping list service to add the item and return the new item's ID


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

# shopping_list.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List all tools
            tools_list = await session.list_tools()
            
            # Display each tool's name, description and input schema
            print("Available tools:\n")
            for tool in tools_list.tools:
                print(f"Name: {tool.name}")
                print(f"Description: {tool.description}")
                print(f"Input Schema:\n{tool.inputSchema}")
            
if __name__ == "__main__":
    asyncio.run(main())
```

## Creating an MCP Tool for a Service

You’ve just seen how to set up an MCP server and learned how to expose service actions as tools with clear documentation. Now it’s your turn to put this into practice.

Your task is to set up the shopping list service inside the MCP server file and create a tool for adding new items. Follow these steps:

Initialize the ShoppingListService so it’s ready to use.
Define the add_item function and register it as an MCP tool. The function should take two parameters: name (a string representing the item's name) and quantity (an integer representing how many to add).
Write a clear docstring for the tool. Be sure to explain what the tool does, describe its parameters and state what it returns (the new item’s ID).
In the function, call the shopping list service to add the item and return the new item’s ID.
When the server starts, it should print out all registered tools so you can see that your tool is discoverable.

```python
# mcp_server.py

# shopping_list.py

# main.py
```

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# TODO: Initialize the shopping list service


# TODO: Define the add_item function and register it as an MCP tool
    # TODO: Parameters: name (str), quantity (int)
    # TODO: Write a docstring that explains what this tool does, its parameters, and what it returns
    # TODO: Call the shopping list service to add the item and return the new item's ID


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

# shopping_list.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List all tools
            tools_list = await session.list_tools()
            
            # Display each tool's name, description and input schema
            print("Available tools:\n")
            for tool in tools_list.tools:
                print(f"Name: {tool.name}")
                print(f"Description: {tool.description}")
                print(f"Input Schema:\n{tool.inputSchema}")
            
if __name__ == "__main__":
    asyncio.run(main())
```

## Calling Tools and Handling Responses

You’ve just seen how to set up an MCP server and create a tool for adding items to the shopping list. Now it’s time to try out the client side and see how to actually use this tool in practice.

Your task is to complete the client code so it connects to the MCP server and calls the add_item tool. Here’s what you need to do:

Call the add_item tool with a sample name and quantity (for example, "Bananas" and 3).
Capture the response from the tool call.
Extract the ID of the newly added item from the response.
Print the added item’s ID so you can see that it worked.
This exercise will help you understand how to interact with MCP tools from a client and how to handle the results.

```python
# mcp_server.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # TODO: Call the add_item tool with sample parameters (e.g., name="Bananas", quantity=3)

            # TODO: Extract the ID of the newly added item from the response's content[0].text

            # TODO: Print the added item's ID

if __name__ == "__main__":
    asyncio.run(main())

# shopping_list.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

# main.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False
```

You're ready to complete the client-side interaction\! The key is to use `session.call_tool` with the correct tool name and a dictionary of arguments, then extract the ID from the response.

Here's the completed `main.py` code:

```python
# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            print("--- Calling add_item Tool ---")

            # TODO: Call the add_item tool with sample parameters (e.g., name="Bananas", quantity=3)
            add_item_response = await session.call_tool("add_item", {"name": "Bananas", "quantity": 3})

            # TODO: Extract the ID of the newly added item from the response's content[0].text
            if add_item_response.content:
                added_item_id = add_item_response.content[0].text
                # TODO: Print the added item's ID
                print(f"Added item ID: {added_item_id}")
            else:
                print("Failed to add item or no ID received.")

            print("--- add_item Tool Call Complete ---")

if __name__ == "__main__":
    asyncio.run(main())

```

The `mcp_server.py` and `shopping_list.py` files remain unchanged from their correct versions:

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

```

```python
# shopping_list.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False
```

## Expanding Your Shopping List Server

Now it’s time to expand your server’s abilities by making it possible to remove items and mark them as purchased. Your task is to add two new tools to the MCP server:

remove_item: This tool should allow clients to remove an item from the shopping list by its unique ID.
mark_purchased: This tool should allow clients to mark an item as purchased or not, using the item’s ID and an optional boolean flag (purchased, which defaults to True).
For each tool, be sure to:

Use the correct method from the ShoppingListService.
Write a clear docstring that explains what the tool does, describes its parameters, and states what it returns.
Make sure the tool returns a boolean value to indicate whether the operation was successful.
When you’re done, the server should print out all registered tools, including your new ones. This exercise will help you practice adding new features and writing helpful documentation for your tools.

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


# TODO: Define the remove_item tool
#   - It should take an item_id parameter (string)
#   - It should return a boolean value
#   - Write a docstring that explains what this tool does, its parameter, and what it returns
#   - Call the shopping_list.remove_item() method


# TODO: Define the mark_purchased tool
#   - It should take an item_id parameter (string) and an optional purchased parameter (bool, default True)
#   - It should return a boolean value
#   - Write a docstring that explains what this tool does, its parameters, and what it returns
#   - Call the shopping_list.set_purchased() method


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

# shopping_list.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List all tools
            tools_list = await session.list_tools()
            
            # Display each tool's name, description and input schema
            print("Available tools:\n")
            for tool in tools_list.tools:
                print(f"Name: {tool.name}")
                print(f"Description: {tool.description}")
                print(f"Input Schema:\n{tool.inputSchema}")
                print("-" * 150)
            
if __name__ == "__main__":
    asyncio.run(main())
```

Great job on setting up the `add_item` tool\! Now, let's expand your MCP server by adding the `remove_item` and `mark_purchased` tools. This will give clients more control over the shopping list.

-----

### Updated `mcp_server.py`

Here's the completed `mcp_server.py` with the two new tools and their comprehensive docstrings:

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


# TODO: Define the remove_item tool
@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list by its unique ID.

    Parameters:
      item_id (str): The unique ID of the item to remove.

    Returns:
      bool: True if the item was successfully removed; False otherwise (e.g., if ID not found).
    """
    # Call the shopping_list.remove_item() method
    return shopping_list.remove_item(item_id)


# TODO: Define the mark_purchased tool
@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item in the shopping list as purchased or not purchased.

    Parameters:
      item_id (str): The unique ID of the item to update.
      purchased (bool, optional): Set to True to mark as purchased, False to mark as not purchased. Defaults to True.

    Returns:
      bool: True if the item's purchased status was successfully updated; False otherwise (e.g., if ID not found).
    """
    # Call the shopping_list.set_purchased() method
    return shopping_list.set_purchased(item_id, purchased)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

```

-----

### Remaining Files (Unchanged)

For completeness, here are the `shopping_list.py` and `main.py` files, which remain the same as provided.

**`shopping_list.py`**

```python
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False
```

**`main.py`**

```python
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List all tools
            tools_list = await session.list_tools()
            
            # Display each tool's name, description and input schema
            print("Available tools:\n")
            for tool in tools_list.tools:
                print(f"Name: {tool.name}")
                print(f"Description: {tool.description}")
                print(f"Input Schema:\n{tool.inputSchema}")
                print("-" * 150)
            
if __name__ == "__main__":
    asyncio.run(main())
```

## Fetching and Filtering Shopping List Items

Tools for adding, removing, and updating shopping list items are in place—now it’s time to boost your server by enabling clients to fetch the current list, with an optional filter for purchased status.

Your task is to implement a new MCP tool called fetch_items in your server. This tool should:

Accept an optional parameter called purchased (a boolean, defaulting to None).
Return a list of shopping items, filtered by the purchased status if provided.
Include a clear docstring that explains what the tool does, describes its parameter, and states what it returns.
Call the get_items method from the ShoppingListService to retrieve the data.
Once you’ve added the tool, update your client code to:

Call the fetch_items tool with no filter (to get all items) and print each item in the result.
Optionally, try calling the tool with purchased=True and purchased=False to see how filtering works.
This exercise will help you practice exposing dynamic data through tools and handling optional parameters in your MCP server.

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.

    Parameters:
      item_id (str): The unique ID of the item.

    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.

    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.

    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


# TODO: Implement the fetch_items tool
#   - Use the @mcp.tool() decorator
#   - The function should be called fetch_items and take an optional purchased parameter (bool, default None)
#   - The return type should be list
#   - Write a docstring that explains what the tool does, its parameter, and what it returns
#   - Call shopping_list.get_items(purchased) and return the result


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

# shopping_list.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # TODO: Call the fetch_items tool with no filter (get all items)
            # TODO: Print each item in the returned list

            # TODO (optional): Try calling fetch_items with purchased=True and print the results
            # TODO (optional): Try calling fetch_items with purchased=False and print the results

if __name__ == "__main__":
    asyncio.run(main())

```

It's time to add the `fetch_items` tool and complete the client workflow\! You'll implement the new tool in `mcp_server.py` to allow fetching all or filtered shopping list items, then update `main.py` to call this tool and print the results.

-----

### Updated `mcp_server.py`

Here's the `mcp_server.py` with the new `fetch_items` tool:

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.

    Parameters:
      item_id (str): The unique ID of the item.

    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.

    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.

    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


# TODO: Implement the fetch_items tool
@mcp.tool()
def fetch_items(purchased: bool = None) -> list:
    """
    Retrieve all shopping list items, optionally filtered by purchased status.

    Parameters:
        purchased (bool, optional): Filter items by purchased status.
            None returns all items,
            True returns only purchased items,
            False returns only unpurchased items.

    Returns:
        list: A list of shopping items matching the filter criteria.
    """
    # Call shopping_list.get_items(purchased) and return the result
    return shopping_list.get_items(purchased)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

```

-----

### Updated `main.py`

Here's the `main.py` updated to call the `fetch_items` tool with and without filters:

```python
# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import json # Import json to pretty print the item dictionaries


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            print("--- Fetching Shopping List Items ---")

            # TODO: Call the fetch_items tool with no filter (get all items)
            print("\nFetching all items:")
            all_items_response = await session.call_tool("fetch_items", {})
            # TODO: Print each item in the returned list
            if all_items_response.content:
                for item_str in all_items_response.content:
                    # Assuming item.text contains a JSON string of the item
                    print(json.dumps(json.loads(item_str.text), indent=2))
            else:
                print("No items found.")

            # TODO (optional): Try calling fetch_items with purchased=True and print the results
            print("\nFetching purchased items:")
            purchased_items_response = await session.call_tool("fetch_items", {"purchased": True})
            if purchased_items_response.content:
                for item_str in purchased_items_response.content:
                    print(json.dumps(json.loads(item_str.text), indent=2))
            else:
                print("No purchased items found.")

            # TODO (optional): Try calling fetch_items with purchased=False and print the results
            print("\nFetching unpurchased items:")
            unpurchased_items_response = await session.call_tool("fetch_items", {"purchased": False})
            if unpurchased_items_response.content:
                for item_str in unpurchased_items_response.content:
                    print(json.dumps(json.loads(item_str.text), indent=2))
            else:
                print("No unpurchased items found.")

            print("\n--- Fetching and Filtering Complete ---")

if __name__ == "__main__":
    asyncio.run(main())

```

-----

### Remaining File (Unchanged)

**`shopping_list.py`**

```python
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False
```

## Testing All Shopping List Tools

It’s time to bring everything together and test your server in a real scenario.

Your task is to write a client script that exercises all four tools — add_item, mark_purchased, remove_item, and fetch_items — in a complete test sequence. This will help you see how the tools work together and ensure your server is behaving as expected.

Here’s what you should do:

Call the add_item tool to add "Carrots" with quantity 4.
Save the ID of the newly added item in a variable called added_id.
Print the added item's ID.
Call the mark_purchased tool using added_id to mark the new item as purchased.
Print whether the operation was successful.
Call the remove_item tool using added_id to remove the new item.
Print whether the removal was successful.
This exercise will help you practice integrating multiple server actions and give you confidence that your tools work well together.


```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.
    
    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.
      
    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.
    
    Parameters:
      item_id (str): The unique ID of the item.
      
    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.
    
    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.
      
    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


@mcp.tool()
def fetch_items(purchased: bool = None) -> list:
    """
    Retrieve all shopping list items, optionally filtered by purchased status.
    
    Parameters:
        purchased (bool, optional): Filter items by purchased status.
            None returns all items,
            True returns only purchased items,
            False returns only unpurchased items.
    
    Returns:
        list: A list of shopping items matching the filter criteria.
    """
    return shopping_list.get_items(purchased)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio") 


# shopping_list.py
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            print("=== Initial shopping list ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

            print("\n=== Adding a new item: Carrots (4) ===")
            # TODO: Call the add_item tool to add "Carrots" with quantity 4
            # TODO: Save the ID of the newly added item in a variable called added_id
            # TODO: Print the added item's ID

            print("\n=== Shopping list after adding ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

            print("\n=== Marking the new item as purchased ===")
            # TODO: Call the mark_purchased tool using added_id
            # TODO: Print whether the operation was successful

            print("\n=== Shopping list after marking as purchased ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

            print("\n=== Removing the new item ===")
            # TODO: Call the remove_item tool using added_id
            # TODO: Print whether the removal was successful

            print("\n=== Final shopping list ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

if __name__ == "__main__":
    asyncio.run(main())
```

Excellent\! It's time to consolidate your work and create a comprehensive client script that tests all your shopping list tools.

Here's the completed `main.py` that will walk through adding an item, marking it as purchased, and then removing it, with print statements to show the results at each step.

-----

### Updated `main.py`

```python
# main.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import json # Import json to pretty print the item dictionaries


async def main():
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            print("=== Initial shopping list ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(json.dumps(json.loads(item.text), indent=2))

            print("\n=== Adding a new item: Carrots (4) ===")
            # TODO: Call the add_item tool to add "Carrots" with quantity 4
            add_response = await session.call_tool("add_item", {"name": "Carrots", "quantity": 4})
            # TODO: Save the ID of the newly added item in a variable called added_id
            if add_response.content:
                added_id = add_response.content[0].text
                # TODO: Print the added item's ID
                print(f"Added item ID: {added_id}")
            else:
                added_id = None
                print("Failed to add Carrots.")

            print("\n=== Shopping list after adding ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(json.dumps(json.loads(item.text), indent=2))

            if added_id:
                print(f"\n=== Marking item {added_id} as purchased ===")
                # TODO: Call the mark_purchased tool using added_id
                mark_response = await session.call_tool("mark_purchased", {"item_id": added_id, "purchased": True})
                # TODO: Print whether the operation was successful
                if mark_response.content:
                    print(f"Mark purchased successful: {mark_response.content[0].text}")
                else:
                    print("Mark purchased failed.")
            else:
                print("\nSkipping mark_purchased: item ID not available.")


            print("\n=== Shopping list after marking as purchased ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(json.dumps(json.loads(item.text), indent=2))

            if added_id:
                print(f"\n=== Removing item {added_id} ===")
                # TODO: Call the remove_item tool using added_id
                remove_response = await session.call_tool("remove_item", {"item_id": added_id})
                # TODO: Print whether the removal was successful
                if remove_response.content:
                    print(f"Remove successful: {remove_response.content[0].text}")
                else:
                    print("Remove failed.")
            else:
                print("\nSkipping remove_item: item ID not available.")

            print("\n=== Final shopping list =====")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(json.dumps(json.loads(item.text), indent=2))

if __name__ == "__main__":
    asyncio.run(main())

```

-----

### Remaining Files (Unchanged)

For completeness, here are the `mcp_server.py` and `shopping_list.py` files, which remain the same as provided.

**`mcp_server.py`**

```python
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.
    
    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.
      
    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.
    
    Parameters:
      item_id (str): The unique ID of the item.
      
    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.
    
    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.
      
    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


@mcp.tool()
def fetch_items(purchased: bool = None) -> list:
    """
    Retrieve all shopping list items, optionally filtered by purchased status.
    
    Parameters:
        purchased (bool, optional): Filter items by purchased status.
            None returns all items,
            True returns only purchased items,
            False returns only unpurchased items.
    
    Returns:
        list: A list of shopping items matching the filter criteria.
    """
    return shopping_list.get_items(purchased)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio") 
```

**`shopping_list.py`**

```python
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False
```